In [1]:
import pandas as pd
import io
df2 = pd.read_csv('~/Documents/DeepLearningProjects/harvard_dl_winter2019/Labs/Lab3/ner_dataset.csv',encoding='latin1')
#df2 = pd.read_csv('ner_dataset.csv',encoding='latin1')

In [2]:
df2 = df2.fillna(method="ffill")
df2.tail(10)
words = list(set(df2["Word"].values))
words.append("ENDPAD")
tags = list(set(df2["Tag"].values))
n_words, n_tags = len(words), len(tags)
print('Number of Words : ', len(words), ' and number of tags : ', len(tags))

Number of Words :  35179  and number of tags :  17


In [3]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [4]:
getter = SentenceGetter(df2)
print(getter.get_next())
sentences = getter.sentences

#max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [5]:
max_len = 50
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words - 1)

y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

Using TensorFlow backend.


In [6]:
from keras.utils import to_categorical

y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

## *Differences I Made*
The Bi-LSTM has 100 units and each gives an output value. But the label number is only 17 in total.
I may think there should be a Dense layer to be inserted between the output and LSTM. （Not so reasonable, I just assume)
But acctually the accuracy is not as high as the origial version.

In [12]:
from keras.models import Model, Input
import numpy as np
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=90, input_length=max_len)(input)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50,activation='softmax',use_bias=True))(model)
model = Dropout(0.1)(model)
out = (Dense(n_tags, activation="softmax"))(model)  # softmax output layer

model = Model(input, out)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5, validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Epoch 1/5
38846/38846 [==============================] - 91s 2ms/step - loss: 1.1269 - acc: 0.9311 - val_loss: 0.3895 - val_acc: 0.9328
Epoch 2/5
38846/38846 [==============================] - 96s 2ms/step - loss: 0.2633 - acc: 0.9337 - val_loss: 0.1701 - val_acc: 0.9328
Epoch 3/5
38846/38846 [==============================] - 98s 3ms/step - loss: 0.1605 - acc: 0.9518 - val_loss: 0.1243 - val_acc: 0.9605
Epoch 4/5
38846/38846 [==============================] - 98s 3ms/step - loss: 0.1216 - acc: 0.9655 - val_loss: 0.0977 - val_acc: 0.9753
Epoch 5/5
38846/38846 [==============================] - 96s 2ms/step - loss: 0.0947 - acc: 0.9759 - val_loss: 0.0764 - val_acc: 0.9817


In [13]:
i = 2318
p = model.predict(np.array([X_te[i]]))
t = [tags[k] for k in np.argmax(y_te[i], axis=-1)]
p = np.argmax(p, axis=-1)
print("{:15} ({:5}): {}".format("Word", "True", "Pred"))
for w, true, pred in zip(X_te[i], t, p[0]):
    print("{:15}: ({:5}): {}".format(words[w], true, tags[pred]))

Word            (True ): Pred
Initial        : (O    ): O
results        : (O    ): O
indicate       : (O    ): O
the            : (O    ): O
ruling         : (O    ): O
party          : (O    ): O
won            : (O    ): O
a              : (O    ): O
majority       : (O    ): O
,              : (O    ): O
but            : (O    ): O
opposition     : (O    ): O
groups         : (O    ): O
say            : (O    ): O
the            : (O    ): O
vote           : (O    ): O
was            : (O    ): O
rigged         : (O    ): O
and            : (O    ): O
declared       : (O    ): O
victory        : (O    ): O
.              : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (O    ): O
ENDPAD         : (